In [12]:
import robotic as ry
import time
with open("logger.txt","w") as f:
    f.write("")
try:
    del C
    del bot
except:
    pass

C = ry.Config()
# C.watchFile("throwing1.g")
C.addFile("throwing_bare.g")
qHome = C.getJointState()
C.view()
#time.sleep(30)
C.getFrame('cargo').unLink()
C.addFrame("initial").setPosition([-0.6,2.5,0.3]).setShape(ry.ST.marker,[.3])
C.addFrame("throw").setPosition([0,2,1.2]).setShape(ry.ST.marker,[.3])
C.view()

0

In [13]:
def pre_grasp_komo(C, gripper_name, grasp_frame_name, q0, qHome):
    komo = ry.KOMO(C, 3, 1, 0, True)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e1])
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0) 
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], qHome)

    komo.addObjective([1., 3.], ry.FS.positionRel, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0, 0, 0])  

    # komo.addObjective([1., 3.], ry.FS.scalarProductXX, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0]) 
    komo.addObjective([1., 3.], ry.FS.scalarProductYX, [gripper_name, grasp_frame_name], ry.OT.eq, [1e2], [1]) 
    # komo.addObjective([1., 3.], ry.FS.scalarProductZZ, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0]) 

    komo.addObjective([1., 3.], ry.FS.scalarProductZZ, [grasp_frame_name, gripper_name], ry.OT.eq, [1e1], [1]) 


    ret = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
    print(ret)
    return komo
qHome = C.getJointState()
def post_grasp_komo(C, gripper_name, grasp_frame_name, q0, qHome):
    komo = ry.KOMO(C, 1, 1, 0, True)
    komo.addObjective([], ry.FS.positionDiff, ["l_gripper","initial"], ry.OT.eq, [1e1], [0,0,0])
    komo.addObjective([], ry.FS.scalarProductYZ, ["l_gripper","initial"], ry.OT.eq, [1e1], [-1])
    ret = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
    print(ret)
    return komo
q0 = C.getJointState()
komo0 = pre_grasp_komo(C, 'l_gripper', 'cargo', q0, qHome)
path0 = komo0.getPath()

komo = post_grasp_komo(C, 'l_gripper', 'cargo', q0, qHome)
path = komo.getPath()
bot = ry.BotOp(C, useRealRobot=False)
bot.home(C)
# print(path.shape)
bot.move(path0, [1.])
while bot.getTimeToEnd() > 0:
    bot.sync(C, .1)

#bot.gripperCloseGrasp(ry._left, 'cargo',speed=1000000000000,width=0.04)
bot.gripperClose(ry._left,width=0.06) 
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)
bot.move(path, [3.])
while bot.getTimeToEnd() > 0:
    bot.sync(C, .1)
#C.getFrame('cargo').unLink()
C.attach('l_gripper', 'cargo')

====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
----newton---- initial point f(x):17270.3 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):    11713.8  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):    6805.72  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):    3626.02  ACCEPT
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):     2024.4  ACCEPT
--newton-- it:   5  |Delta|:        0.2  alpha:          1  evals:   6  f(y):    1019.56  ACCEPT
--newton-- it:   6  |Delta|:        0.2  alpha:          1  evals:   7  f(y):     396.06  ACCEPT
--newton-- it:   7  |Delta|:        0.2  alpha:          1  evals:   8  f(y):    76.0648  ACCEPT
--newton-- it:   8  |Delta|:   0.174241  alpha:          1  evals:   9  f(y):    5.82648  ACCEPT
--newton-- it:   9  |Delta|:  0.0891476  alpha:          1  evals:  10  f(y

In [14]:
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import differential_evolution
from my_utils import rotation_matrix_to_quaternion, velocity_to_rotation_matrix, find_line_intersections_2d

In [15]:
import numpy as np
from scipy.optimize import minimize

# Define constants
g = 9.81  # Gravity, m/s^2

# Initial positions (robot and bin)
initial_pos = C.getFrame("throw").getPosition()
bin_dims = C.getFrame("bin").getPosition()

# Bin and robot parameters
x_bin, y_bin, z_bin = bin_dims[0], bin_dims[1], bin_dims[2]
x0, y0, z0 = initial_pos[0], initial_pos[1], initial_pos[2]

# Objective function: minimize velocity magnitude
def objective(params):
    v0, theta, phi = params
    return v0

# Constraints: ball lands in the bin's X position
def constraint_x(params):
    v0, theta, phi = params
    vx = v0 * np.cos(theta) * np.cos(phi)
    t_land = (-v0 * np.sin(theta) + np.sqrt((v0 * np.sin(theta))**2 + 2 * g * (z0 - z_bin))) / g
    return x0 + vx * t_land - x_bin

# Constraints: ball lands in the bin's Y position
def constraint_y(params):
    v0, theta, phi = params
    vy = v0 * np.cos(theta) * np.sin(phi)
    t_land = (-v0 * np.sin(theta) + np.sqrt((v0 * np.sin(theta))**2 + 2 * g * (z0 - z_bin))) / g
    return y0 + vy * t_land - y_bin

# Constraints: ball lands in the bin's Z position (height)
def constraint_z(params):
    v0, theta, phi = params
    vz = v0 * np.sin(theta)
    t_land = (-vz + np.sqrt(vz**2 + 2 * g * (z0 - z_bin))) / g
    return z0 + vz * t_land - 0.5 * g * t_land**2 - z_bin

# Constraint: Ball does not hit the bin's sides
def constraint_sides_height(params):
    v0, theta, phi = params
    # Time to reach maximum height
    t_up = v0 * np.sin(theta) / g
    # Ball's height at time t_up (maximum height)
    z_max = z0 + (v0 * np.sin(theta))**2 / (2 * g)
    
    # If the bin's sides are higher than the max height, no problem
    if z_max < z_bin:
        return 0  # No issue, the ball won't hit the sides
    else:
        # If the bin's sides are lower, we need to check at different points in time
        t_down = np.sqrt(2 * (z_max - z_bin) / g)
        z_ball_at_t = z0 + v0 * np.sin(theta) * t_down - 0.5 * g * t_down**2
        return z_ball_at_t - z_bin  # Ball height should not exceed bin sides

# Initial guess and bounds
initial_guess = [5, np.pi / 4, np.pi / 4]  # [velocity, theta, phi]
bounds = [(1, 20), (0, np.pi / 2), (0, 2 * np.pi)]  # Bounds for velocity and angles

# Solve the optimization
result = minimize(objective, initial_guess, constraints=[
    {'type': 'eq', 'fun': constraint_x},
    {'type': 'eq', 'fun': constraint_y},
    {'type': 'eq', 'fun': constraint_z},
    {'type': 'ineq', 'fun': constraint_sides_height}  # Ensure ball avoids sides' height
], bounds=bounds)

# Extract results
v0_opt, theta_opt, phi_opt = result.x
print(f"Optimal velocity: {v0_opt:.2f} m/s, Theta: {np.degrees(theta_opt):.2f}°, Phi: {np.degrees(phi_opt):.2f}°")


Optimal velocity: 4.19 m/s, Theta: 0.00°, Phi: 0.00°


In [16]:
print("Velocity:", v0_opt)
print("Release Angle theta (degrees):", theta_opt)
print("Release Angle phi (degrees):", phi_opt)

v_x = v0_opt * np.cos(theta_opt) * np.cos(phi_opt)
v_y = v0_opt * np.cos(theta_opt) * np.sin(phi_opt)
v_z = v0_opt * np.sin(theta_opt)

velocity = [v_x, v_y, v_z]

C.addFrame("release_frame").setPosition(initial_pos).setShape(ry.ST.marker,[.4]).setColor([1,0,0]).setContact(0)


C.view()
#finding the initial point from the release point and the velocity
#find the backward kinematics and simulate it


def find_initial_point_from_release(release_position, release_velocity, robot_base, height_threshold=0.2, distance_threshold=1.53):
   """
   Finds the initial position for the throwing motion based on the release position and velocity.


   Args:
       C: Robot configuration (KOMO simulation object).
       release_position: The release point (3D coordinates).
       release_velocity: The release velocity vector.
       robot_base: Position of the robot base.
       height_threshold: Minimum allowable height for the initial position.
       distance_threshold: Maximum allowable distance from the robot base.


   Returns:
       initial_position: The computed initial position, if found.
   """
   # Compute the initial position using backward kinematics
   time_step = 0.01  # Small time step for simulation
   gravity = np.array([0, 0, -9.81])  # Gravity vector


   # Backward integrate to find the initial position
   velocity = np.array(release_velocity)
   position = np.array(release_position)
   distance_to_base = np.linalg.norm(position - np.array(robot_base))
   iteration = 0
   prev_pos = position
   while position[2] > height_threshold and distance_to_base <= distance_threshold:  # Ensure height is above threshold
       iteration+=1
       prev_pos = position
       position -= velocity * time_step
       velocity -= gravity * time_step
       # Check distance to the robot base
       distance_to_base = np.linalg.norm(position - np.array(robot_base))
       #print(f"Iter {iteration}")
   iteration = 0
   velocity = np.array(release_velocity)
   time_step = .1
   position = np.array(release_position)


   while position[2] > height_threshold:
       position += velocity * time_step
       velocity += gravity * time_step
       iteration += 1
       frame = C.addFrame(f"trajectory-{iteration}").setShape(ry.ST.marker,[.2]).setColor([0,0,1]).setPosition(position)
       C.view()
       print(f"Frame {iteration} added with position {frame.getPosition()} new vel:{velocity}")
   return prev_pos  # No valid initial position found


def pick_last_object_if_valid(release_position, release_velocity, robot_base):
   """
   Simulates the robot to pick the last object if conditions are met.


   Args:
       C: Robot configuration (KOMO simulation object).
       release_position: The release point (3D coordinates).
       release_velocity: The release velocity vector.
       robot_base: Position of the robot base.


   Returns:
       None
   """
   # Find the initial position
   initial_position = find_initial_point_from_release(release_position, release_velocity, robot_base)


   if initial_position is not None:
       print(f"Initial Position Found: {initial_position}")
       # Visualize the initial position
       C.addFrame("initial_position").setPosition(initial_position).setShape(ry.ST.marker, [0.4]).setContact(0).setColor([0,1,0]).setQuaternion(rotation_matrix_to_quaternion(velocity_to_rotation_matrix(release_velocity)))
       C.view()
       return initial_position
       # Simulate the robot to pick the last object
       # Add your grasping or motion planning logic here
   else:
       print("No valid initial position found.")


initial_pos = pick_last_object_if_valid(np.array(initial_pos),np.array(velocity),C.getFrame("l_panda_base").getPosition())


#go to inital position with y axis being scalar to the velocity vector
def goto_init(gripper_name, initial_pos_name):
   komo = ry.KOMO(C, 1, 1, 0, True)
   komo.addObjective([1.], ry.FS.positionDiff, [gripper_name, initial_pos_name], ry.OT.eq, [1e1], [0, 0, 0])
   komo.addObjective([1], ry.FS.scalarProductYZ, [gripper_name, initial_pos_name], ry.OT.eq, [1e1], [-1])
   ret2 = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
   print(ret2)
   return komo

Velocity: 4.185433897238976
Release Angle theta (degrees): 1.1102230246251565e-16
Release Angle phi (degrees): 0.0
Frame 1 added with position [0.41854339 2.         1.2       ] new vel:[ 4.1854339  0.        -0.981    ]
Frame 2 added with position [0.83708678 2.         1.1019    ] new vel:[ 4.1854339  0.        -1.962    ]
Frame 3 added with position [1.25563017 2.         0.9057    ] new vel:[ 4.1854339  0.        -2.943    ]
Frame 4 added with position [1.67417356 2.         0.6114    ] new vel:[ 4.1854339  0.        -3.924    ]
Frame 5 added with position [2.09271695 2.         0.219     ] new vel:[ 4.1854339  0.        -4.905    ]
Frame 6 added with position [ 2.51126034  2.         -0.2715    ] new vel:[ 4.1854339  0.        -5.886    ]
Initial Position Found: [-1.42304753  2.          0.649659  ]


In [17]:
# Add frame for the release position
release_frame = C.addFrame('release')
release_frame.setShape(ry.ST.marker, size=[0])
release_frame.setPosition(initial_pos)
C.view()
time.sleep(15)

In [18]:
#code for calculating velocity
def vel_komo():
    q0 = C.getJointState()
    komo = ry.KOMO(C, 1, 1, 1, True)
    komo.addObjective([], ry.FS.position, ["l_gripper"], ry.OT.eq, [1e-1], [4.2,-3.5,4],1)
    komo.addObjective([],ry.FS.positionDiff,["l_gripper","throw"],ry.OT.sos,[1e0],[0,0,0])
    ret2 = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
    #komo.addObjective([1.],ry.FS.scalarProductXZ,["base","l_gripper"],ry.OT.eq,[1e1],[-1],0)
    #print(komo.report())
    print(f"ret!!!: {ret2}")
    return komo
komo = vel_komo()
# bot = ry.BotOp(C,False)
# time.sleep(5)
path = komo.getPath()
print(f"path size: {path.size}")
bot.move(path,[1.])
time.sleep(0.55)
#print(f"bot end time:{bot.getTimeToEnd()}")
bot.gripperMove(ry._left,width=1)
# while bot.gripperDone(ry._left):
#     bot.sync(C,.1)
print(f"bot end time:{bot.getTimeToEnd()}")
while bot.getTimeToEnd()>0:
    bot.sync(C,.1)


====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
----newton---- initial point f(x):1.81736 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):    1.36924  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):    1.03102  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):   0.779074  ACCEPT
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):   0.594183  ACCEPT
--newton-- it:   5  |Delta|:        0.2  alpha:          1  evals:   6  f(y):   0.463665  ACCEPT
--newton-- it:   6  |Delta|:        0.2  alpha:          1  evals:   7  f(y):   0.380259  ACCEPT
--newton-- it:   7  |Delta|:   0.128235  alpha:          1  evals:   8  f(y):   0.348058  ACCEPT
--newton-- it:   8  |Delta|:  0.0574086  alpha:          1  evals:   9  f(y):   0.336813  ACCEPT
--newton-- it:   9  |Delta|:  0.0460998  alpha:          1  evals:  10  f(y

bot end time:0.9500000000000011-- kin_physx.cpp:removeJoint:891(0) REMOVING JOINT 0x5bb043bff840-0x5bb041a3a690 of type rigid



In [19]:
time.sleep(4)
del C
del bot

-- bot.cpp:~BotOp:135(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:57(0) shutting down SimThread
-- simulation.cpp:~Simulation:148(0) shutting down Simulation
